# Data cleaning Immobiliare.it

Importing the librares

In [82]:
import numpy as np
import pandas as pd
import re

I check the raw dataframe and the datatypes

In [83]:
df_raw = pd.read_csv('house_prices_italy.csv')
df_raw.head()

,Unnamed: 0,region,city,area,rooms,toilets,price
0,0,abruzzo,Pescara,89m²,3,1,€ 75.000
1,1,abruzzo,Spoltore,199m²,5+,3+,€ 235.000
2,2,abruzzo,Pescara,227m²,5,3+,€ 299.000
3,3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43m²,2 - 4,1,da € 165.000
4,4,abruzzo,Roseto degli Abruzzi,978m²,5+,3+,€ 1.500.000


In [84]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  40000 non-null  int64 
 1   region      40000 non-null  object
 2   city        40000 non-null  object
 3   area        39975 non-null  object
 4   rooms       39050 non-null  object
 5   toilets     40000 non-null  object
 6   price       40000 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.1+ MB


Are there any null values? let's see:

In [85]:
df_raw.isnull().sum()

Unnamed: 0      0
region          0
city            0
area           25
rooms         950
toilets         0
price           0
dtype: int64

I decided to drop the first column since it does not carry any info:

In [86]:
df=df_raw.drop("Unnamed: 0", axis=1)

In [87]:
df.head()

,region,city,area,rooms,toilets,price
0,abruzzo,Pescara,89m²,3,1,€ 75.000
1,abruzzo,Spoltore,199m²,5+,3+,€ 235.000
2,abruzzo,Pescara,227m²,5,3+,€ 299.000
3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43m²,2 - 4,1,da € 165.000
4,abruzzo,Roseto degli Abruzzi,978m²,5+,3+,€ 1.500.000


I force the "area" column as 'string', if not, later it will give errors.

In [88]:
df['area'] = df['area'].astype(str)

I do not like the column names. I rename them:

In [89]:
df.rename(columns={'area':'area[m2]', 'price':'price[€]'}, inplace=True)
df.sample(10)

,region,city,area[m2],rooms,toilets,price[€]
6837,calabria,Botricello,225m²,5+,3,€ 89.000
16337,lombardia,Attici e Appartamenti di nuova costruzione a M...,59m²,2 - 3,1,da € 296.000
28682,sicilia,Pantelleria,74m²,2,1,€ 180.000
3303,basilicata,Pisticci,130m²,5,2,€ 110.000
22899,piemonte,Pianezza,204m²,5,3,€ 307.000
26472,sardegna,Sant'Antonio di Gallura,339m²,5+,2,€ 130.000
17850,lombardia,Gessate,308m²,5+,3,€ 535.000
10273,friuli-venezia-giulia,Lignano Sabbiadoro,95m²,3,1,€ 275.000
2078,basilicata,Maratea,160m²,4,2,€ 169.000
26976,sardegna,Pauli Arbarei,450m²,5+,1,€ 300.000


Regex on "area" column. Let's start cleaning from here:

In [90]:
#compilo il pattern
p = re.compile('[0-9]+')


df['area[m2]'] = df['area[m2]'].apply(lambda x: np.nan if p.search(x) is None else p.search(x).group())
df.head()

,region,city,area[m2],rooms,toilets,price[€]
0,abruzzo,Pescara,89,3,1,€ 75.000
1,abruzzo,Spoltore,199,5+,3+,€ 235.000
2,abruzzo,Pescara,227,5,3+,€ 299.000
3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43,2 - 4,1,da € 165.000
4,abruzzo,Roseto degli Abruzzi,978,5+,3+,€ 1.500.000


In [91]:
df.isna().sum()

region        0
city          0
area[m2]     25
rooms       950
toilets       0
price[€]      0
dtype: int64

Now it's the turn for the "price" column:

In [92]:
p_price = re.compile('[0-9]+\.[0-9]+')

In [93]:
df['price[€]'] = df['price[€]'].apply(lambda x: np.nan if p_price.search(x) is None else p_price.search(x).group()).str.replace('.','', regex=False)
df.isna().sum()

region         0
city           0
area[m2]      25
rooms        950
toilets        0
price[€]    1254
dtype: int64

I now rename the regions into a more appropriate manner:

In [94]:
regions_dict = {'abruzzo':'Abruzzo', 'basilicata':'Basilicata', 'campania':'Campania', 'calabria':'Calabria', 'emilia-romagna':'Emilia Romagna',
       'friuli-venezia-giulia':'Friuli-Venezia Giulia', 'lazio': 'Lazio', 'liguria':'Liguria', 'lombardia':'Lombardia', 'marche':'Marche',
       'molise':'Molise', 'piemonte':'Piemonte', 'puglia':'Puglia', 'sardegna':'Sardegna', 'sicilia':'Sicilia', 'toscana':'Toscana',
       'trentino-alto-adige': 'Trentino-Alto Adige', 'umbria':'Umbria', 'valle-d-aosta':'Valle d\'Aosta', 'veneto':'Veneto'}

df.replace({'region':regions_dict}, inplace=True)
df.isna().sum()


region         0
city           0
area[m2]      25
rooms        950
toilets        0
price[€]    1254
dtype: int64

In [95]:
df.sample(20)

,region,city,area[m2],rooms,toilets,price[€]
1989,Abruzzo,L'Aquila,170,2,2,255000
10657,Friuli-Venezia Giulia,Savogna,150,5,1,300000
8288,Emilia Romagna,Bologna,140,5,2,539000
4627,Campania,Napoli,140,4,2,395000
36132,Valle d'Aosta,Montjovet,80,3,1,98000
5496,Campania,Napoli,110,3,1,175000
7927,Calabria,Scalea,700,5+,3+,159000
5349,Campania,Capri,100,4,2,895000
36181,Valle d'Aosta,Aymavilles,75,2,1,80000
20036,Molise,Isernia,106,3,1,93000


I believe this is enough. It will be possible to load the data into a DataViz software and keep going from there with some visualizations

In [96]:
df.to_csv('house_prices_italy_cleaned.csv')